# 학번: 20901     이름: 경승민
# CAPTCHA 스타일 이미지 타일 인식 시스템

## 1. 라이브러리 불러오기

In [ ]:
# 필요한 라이브러리 불러오기
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
import os

## 2. 데이터 준비 및 전처리
### CIFAR-10 데이터셋 사용 (10개 클래스: 비행기, 자동차, 새, 고양이, 사슴, 개, 개구리, 말, 배, 트럭)

In [ ]:
# CIFAR-10 데이터셋 불러오기 (케라스에서 지원)
cifar10 = keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# 클래스 이름 정의
class_names = ['비행기', '자동차', '새', '고양이', '사슴', '개', '개구리', '말', '배', '트럭']

print(f'훈련 데이터 크기: {x_train.shape}')
print(f'테스트 데이터 크기: {x_test.shape}')

In [ ]:
# 훈련 데이터 샘플 확인하기
plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i+1)
    plt.imshow(x_train[i])
    plt.title(class_names[y_train[i][0]])
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# 데이터 정규화 (0~255를 0.0~1.0으로 변환)
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# y 데이터를 1차원으로 변환
y_train = y_train.flatten()
y_test = y_test.flatten()

print(f'정규화된 데이터 범위: {x_train.min()} ~ {x_train.max()}')

## 3. CNN 모델 생성하기

In [ ]:
# CNN 모델 구성하기
model = keras.models.Sequential([
    # 첫 번째 합성곱 레이어
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    
    # 두 번째 합성곱 레이어
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    # 세 번째 합성곱 레이어
    layers.Conv2D(64, (3, 3), activation='relu'),
    
    # Flatten 레이어 (1차원으로 펼치기)
    layers.Flatten(),
    
    # 완전 연결 레이어 (Dense)
    layers.Dense(64, activation='relu'),
    
    # 출력 레이어 (10개 클래스)
    layers.Dense(10, activation='softmax')
])

# 모델 구조 확인
model.summary()

In [ ]:
# 모델 컴파일하기
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

## 4. 모델 학습하기

In [ ]:
# 모델 학습 (epochs: 반복 학습 횟수)
history = model.fit(
    x_train, y_train,
    epochs=10,
    validation_data=(x_test, y_test),
    batch_size=64
)

In [ ]:
# 학습 과정 시각화
plt.figure(figsize=(12, 4))

# 정확도 그래프
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='훈련 정확도')
plt.plot(history.history['val_accuracy'], label='검증 정확도')
plt.xlabel('Epoch')
plt.ylabel('정확도')
plt.legend()
plt.title('모델 정확도')

# 손실 그래프
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='훈련 손실')
plt.plot(history.history['val_loss'], label='검증 손실')
plt.xlabel('Epoch')
plt.ylabel('손실')
plt.legend()
plt.title('모델 손실')

plt.tight_layout()
plt.show()

## 5. 모델 평가하기

In [ ]:
# 테스트 데이터로 모델 평가
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f'테스트 정확도: {test_accuracy:.4f}')
print(f'테스트 손실: {test_loss:.4f}')

## 6. CAPTCHA 타일 시스템 구현
### 9개의 타일 중 특정 객체가 포함된 타일 찾기

In [ ]:
# 이미지를 9개의 타일로 분할하는 함수
def split_image_to_tiles(image, tile_size=32):
    """
    이미지를 3x3 그리드로 분할
    
    Parameters:
    - image: 입력 이미지 (96x96 크기 권장)
    - tile_size: 각 타일의 크기 (기본값 32x32)
    
    Returns:
    - tiles: 9개의 타일 배열
    """
    tiles = []
    
    # 3x3 그리드로 분할
    for i in range(3):
        for j in range(3):
            # 타일 추출
            tile = image[i*tile_size:(i+1)*tile_size, 
                        j*tile_size:(j+1)*tile_size]
            tiles.append(tile)
    
    return np.array(tiles)

# 타일 시각화 함수
def visualize_tiles(tiles, predictions=None, target_class=None):
    """
    9개의 타일을 시각화
    
    Parameters:
    - tiles: 타일 배열
    - predictions: 각 타일의 예측 결과 (선택사항)
    - target_class: 찾고자 하는 타겟 클래스 (선택사항)
    """
    plt.figure(figsize=(10, 10))
    
    for i in range(9):
        plt.subplot(3, 3, i+1)
        plt.imshow(tiles[i])
        
        if predictions is not None:
            pred_class = np.argmax(predictions[i])
            confidence = predictions[i][pred_class] * 100
            
            # 타겟 클래스와 일치하면 초록색 테두리
            if target_class is not None and pred_class == target_class:
                plt.gca().add_patch(plt.Rectangle((0, 0), 31, 31, 
                                                  fill=False, 
                                                  edgecolor='green', 
                                                  linewidth=3))
            
            plt.title(f'{class_names[pred_class]}\n({confidence:.1f}%)')
        
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# CAPTCHA 타일 분류 함수
def classify_tiles(tiles, model, target_class):
    """
    9개의 타일을 분류하고 타겟 클래스가 포함된 타일 찾기
    
    Parameters:
    - tiles: 타일 배열
    - model: 학습된 CNN 모델
    - target_class: 찾고자 하는 클래스 (0~9)
    
    Returns:
    - predictions: 각 타일의 예측 결과
    - target_tiles: 타겟 클래스가 포함된 타일의 인덱스 리스트
    """
    # 모든 타일 예측
    predictions = model.predict(tiles)
    
    # 타겟 클래스가 포함된 타일 찾기
    target_tiles = []
    
    for i in range(len(tiles)):
        pred_class = np.argmax(predictions[i])
        
        if pred_class == target_class:
            target_tiles.append(i)
    
    return predictions, target_tiles

## 7. CAPTCHA 시스템 테스트

In [ ]:
# 테스트용 96x96 이미지 생성 (9개의 랜덤 이미지 조합)
def create_captcha_image(x_data, y_data, target_class=None, num_targets=2):
    """
    9개의 타일로 구성된 CAPTCHA 이미지 생성
    
    Parameters:
    - x_data: 이미지 데이터
    - y_data: 라벨 데이터
    - target_class: 포함시킬 타겟 클래스 (선택사항)
    - num_targets: 타겟 클래스를 몇 개 포함시킬지 (기본값 2)
    
    Returns:
    - captcha_image: 96x96 크기의 CAPTCHA 이미지
    - true_labels: 각 타일의 실제 라벨
    """
    captcha_image = np.zeros((96, 96, 3), dtype='float32')
    true_labels = []
    
    # 타겟 클래스가 지정된 경우
    if target_class is not None:
        # 타겟 클래스 이미지 인덱스 찾기
        target_indices = np.where(y_data == target_class)[0]
        other_indices = np.where(y_data != target_class)[0]
        
        # 타겟 타일 위치 랜덤 선택
        target_positions = np.random.choice(9, num_targets, replace=False)
        
        tile_idx = 0
        for i in range(3):
            for j in range(3):
                # 타겟 위치인 경우
                if tile_idx in target_positions:
                    img_idx = np.random.choice(target_indices)
                else:
                    img_idx = np.random.choice(other_indices)
                
                # 이미지 삽입
                captcha_image[i*32:(i+1)*32, j*32:(j+1)*32] = x_data[img_idx]
                true_labels.append(y_data[img_idx])
                
                tile_idx += 1
    else:
        # 랜덤하게 9개 선택
        random_indices = np.random.choice(len(x_data), 9, replace=False)
        
        tile_idx = 0
        for i in range(3):
            for j in range(3):
                img_idx = random_indices[tile_idx]
                captcha_image[i*32:(i+1)*32, j*32:(j+1)*32] = x_data[img_idx]
                true_labels.append(y_data[img_idx])
                tile_idx += 1
    
    return captcha_image, np.array(true_labels)

In [ ]:
# CAPTCHA 이미지 생성 (자동차를 2개 포함)
target_class = 1  # 자동차 클래스
captcha_img, true_labels = create_captcha_image(x_test, y_test, 
                                                 target_class=target_class, 
                                                 num_targets=2)

# CAPTCHA 이미지 시각화
plt.figure(figsize=(8, 8))
plt.imshow(captcha_img)
plt.title(f'CAPTCHA 이미지: {class_names[target_class]}를 모두 선택하세요')
plt.axis('off')
plt.show()

print(f'실제 라벨: {[class_names[label] for label in true_labels]}')

In [ ]:
# 타일로 분할
tiles = split_image_to_tiles(captcha_img, tile_size=32)

print(f'타일 개수: {len(tiles)}')
print(f'타일 크기: {tiles[0].shape}')

In [ ]:
# 타일 분류 및 타겟 찾기
predictions, target_tiles = classify_tiles(tiles, model, target_class)

print(f'찾고자 하는 객체: {class_names[target_class]}')
print(f'감지된 타일 인덱스: {target_tiles}')
print(f'실제 정답 타일 인덱스: {np.where(true_labels == target_class)[0].tolist()}')

In [ ]:
# 결과 시각화 (타겟 클래스가 포함된 타일에 초록색 테두리)
visualize_tiles(tiles, predictions, target_class)

## 8. 다양한 케이스 테스트

In [ ]:
# 여러 객체로 테스트
test_classes = [0, 1, 2, 3]  # 비행기, 자동차, 새, 고양이

for test_class in test_classes:
    print(f'\n=== {class_names[test_class]} 찾기 ===')
    
    # CAPTCHA 이미지 생성
    captcha_img, true_labels = create_captcha_image(x_test, y_test, 
                                                     target_class=test_class, 
                                                     num_targets=3)
    
    # 타일 분할
    tiles = split_image_to_tiles(captcha_img, tile_size=32)
    
    # 분류
    predictions, target_tiles = classify_tiles(tiles, model, test_class)
    
    print(f'감지된 타일: {target_tiles}')
    print(f'실제 정답: {np.where(true_labels == test_class)[0].tolist()}')
    
    # 시각화
    visualize_tiles(tiles, predictions, test_class)

## 9. 커스텀 이미지로 테스트 (선택사항)
### 자신의 이미지를 업로드하여 테스트

In [ ]:
# 커스텀 이미지 로드 및 전처리 함수
def load_custom_image(image_path, target_size=(96, 96)):
    """
    커스텀 이미지를 로드하고 전처리
    
    Parameters:
    - image_path: 이미지 파일 경로
    - target_size: 타겟 크기 (기본값 96x96)
    
    Returns:
    - processed_image: 전처리된 이미지
    """
    # 이미지 읽기
    img = cv2.imread(image_path)
    
    # BGR to RGB 변환
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # 크기 조정
    img = cv2.resize(img, target_size)
    
    # 정규화
    img = img.astype('float32') / 255.0
    
    return img

# 사용 예시 (이미지 파일이 있는 경우)
# custom_img = load_custom_image('captcha.png')
# custom_tiles = split_image_to_tiles(custom_img, tile_size=32)
# custom_predictions, custom_target_tiles = classify_tiles(custom_tiles, model, target_class=1)
# visualize_tiles(custom_tiles, custom_predictions, target_class=1)

## 10. 정확도 평가

In [ ]:
# CAPTCHA 시스템 정확도 평가
def evaluate_captcha_system(model, x_data, y_data, num_tests=100):
    """
    CAPTCHA 시스템의 정확도 평가
    
    Parameters:
    - model: 학습된 모델
    - x_data: 테스트 데이터
    - y_data: 테스트 라벨
    - num_tests: 테스트 횟수
    
    Returns:
    - accuracy: 정확도
    """
    correct = 0
    total = 0
    
    for _ in range(num_tests):
        # 랜덤 타겟 클래스 선택
        target_class = np.random.randint(0, 10)
        
        # CAPTCHA 생성
        captcha_img, true_labels = create_captcha_image(x_data, y_data, 
                                                         target_class=target_class, 
                                                         num_targets=2)
        
        # 타일 분할 및 분류
        tiles = split_image_to_tiles(captcha_img, tile_size=32)
        predictions, predicted_tiles = classify_tiles(tiles, model, target_class)
        
        # 실제 정답
        actual_tiles = np.where(true_labels == target_class)[0].tolist()
        
        # 정확도 계산 (정확히 일치하는 경우)
        if set(predicted_tiles) == set(actual_tiles):
            correct += 1
        
        total += 1
    
    accuracy = correct / total
    return accuracy

# 정확도 평가
captcha_accuracy = evaluate_captcha_system(model, x_test, y_test, num_tests=50)
print(f'\nCAPTCHA 시스템 정확도: {captcha_accuracy * 100:.2f}%')

## 11. 모델 저장 및 로드

In [ ]:
# 모델 저장하기
model.save('captcha_tile_model.h5')
print('모델이 저장되었습니다.')

In [ ]:
# 저장된 모델 불러오기
# loaded_model = keras.models.load_model('captcha_tile_model.h5')
# print('모델이 로드되었습니다.')